Creating Match Projections using OOP

In [1]:
import pandas as pd
import numpy as np

from data_preparation import load_match_summary
from data_preparation import load_player_stats
from data_preparation import aggregate_player_to_match_stats
from data_preparation import load_team_info
from data_preparation import load_venue_info
from data_preparation import create_match_summary_stats

from player import Player
from match import Match
from team import Team
from rating_calculator import PlayerRatingCalculator, TeamRatingCalculator
from projector import Projector

from match_projections import initialise_teams_players, update_ratings, actual_vs_expected

import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.precision', 4)
%load_ext autoreload
%autoreload 2

1. load expected vaep data and player data
2. simulate initial updating process
3. put into a function
4. create objective function
5. run optuna optimisation to find update parameters

Load in player and match stats

In [2]:
match_summary = load_match_summary()
player_stats = load_player_stats()
match_stats = aggregate_player_to_match_stats(player_stats)
team_info, home_team_info, away_team_info = load_team_info()
venue_info, away_venue_info = load_venue_info()
match_summary_stats = create_match_summary_stats(match_summary, match_stats, home_team_info, away_team_info, away_venue_info)

Updating Process:

1. Sum up projected player values from each game as the projected rating for that team
2. Get difference in match expected vaep values as the actual rating for that match
3. Take into account home advantage for home team
4. Update term player ratings based on actual v projected value

In [113]:
round_id_list = ['2020F4', '202101', '202102', '2022F1', '202304', '202105', '2023F2']

In [114]:
def get_latest_available_round_id(round_id_list):
    latest_available_season = np.max(np.array([int(x[:4]) for x in round_id_list]))
    latest_season_round_id_list = [x for x in round_id_list if str(latest_available_season) in x]
    latest_season_round_list = [x[4:] for x in latest_season_round_id_list]
    
    if latest_available_season == 2023:
        finals_round_map = {
            'F1':25,
            'F2':26,
            'F3':27,
            'F4':28}
        reverse_finals_round_map ={
            25:'F1',
            26:'F2',
            27:'F3',
            28:'F4'
        }
        last_round = 24
    else:
        finals_round_map = {
            'F1':24,
            'F2':25,
            'F3':26,
            'F4':27}
        reverse_finals_round_map ={
            24:'F1',
            25:'F2',
            26:'F3',
            27:'F4'
        }
        last_round = 23
            
    latest_season_round_num_list = [int(finals_round_map[x]) if 'F' in x else int(x) for x in latest_season_round_list]           

    latest_season_round_num = np.max(np.array(latest_season_round_num_list))
    if latest_season_round_num > last_round:
        latest_season_round = reverse_finals_round_map[latest_season_round_num]
    else:
        latest_season_round = str(latest_season_round_num).zfill(2)
    latest_available_round_id = str(latest_available_season) + str(latest_season_round)
    return latest_available_round_id

In [115]:
get_latest_available_round_id(round_id_list)

'2023F2'

## Initialise Players

* Players have ratings
* Teams are made up of players in the squad for a given match (teams have players, players don't have teams)
* Matches are two teams playing each other at a specific venue
* Teams ratings are the sum of their players ratings for a given match (teams only have ratings in the context of a match)
* Rounds are made up of matches
* Seasons are made up of rounds

In [3]:
from dataclasses import dataclass, field, make_dataclass
from typing import List, Dict, Optional

@dataclass
class Player:
    name: str
    rating: Dict = field(default_factory=lambda: {})
    
    def __repr__(self) -> str:
        return 'Player({})'.format(self.name)
    
    def get_rating(self, round_id):
        try:
            return self.rating[round_id]
        except:
            raise KeyError("Round {} not available.".format(round_id)) from None
        
    def get_latest_available_rating(self):
        available_rounds = list(self.rating.keys())
            
    def add_rating(self, round_id, rating_value):
        self.rating[round_id] = rating_value 
    
@dataclass
class Team:
    name: str
    players: Dict[str, Player]
    
    def __repr__(self) -> str:
        return 'Team({})'.format(self.name)
    
    def get_player_ratings(self) -> pd.DataFrame:
        return pd.DataFrame().from_dict({k: v.rating for (k, v) in self.players.items()}, orient = 'index')
        
    
@dataclass
class Match:
    match_id: str
    home_team: Team
    away_team: Team
    venue: str
    match_summary: Optional[pd.DataFrame] = None
    player_stats: Optional[pd.DataFrame] = None
    
    def __post_init__(self):
    
        if self.match_summary is not None:
            self.match_summary = self.match_summary[self.match_summary['Match_ID'] == self.match_id]
        if self.player_stats is not None:
            self.player_stats = self.player_stats[self.player_stats['Match_ID'] == self.match_id]
            self.match_stats = self.aggregate_player_to_match_stats()
        self.season = int(self.match_id.split("_")[0][:4])
        self.round_id = self.match_id.split("_")[0]
        self.previous_round_id = self.get_previous_round_id()
        
    def __repr__(self) -> str:
        return 'Match({} v {} @ {} in {})'.format(self.home_team.name, self.away_team.name, self.venue, self.round_id)
        
    def aggregate_player_to_match_stats(self) -> pd.DataFrame:
    
        self.match_stats = self.player_stats.groupby(['Match_ID', 'Team', 'Home_Team']).sum()[['Score', 'xScore', 'exp_vaep_value', 'exp_offensive_value', 'exp_defensive_value']].reset_index()
        self.match_stats['Home_Away'] = np.where(self.match_stats['Team'] == self.match_stats['Home_Team'], 'Home', 'Away')
        self.match_stats = self.match_stats.pivot(index = 'Match_ID', columns = 'Home_Away', values = ['Score', 'xScore', 'exp_vaep_value', 'exp_offensive_value', 'exp_defensive_value'])
        self.match_stats.columns = ['_'.join((y, x)) for (x, y) in self.match_stats.columns]
        
        return self.match_stats
        
    def get_previous_round_id(self):
        
        season = int(self.round_id[:4])
        if season == 2023:
            finals_round_map = {
                'F1':25,
                'F2':26,
                'F3':27,
                'F4':28}
            reverse_finals_round_map ={
                25:'F1',
                26:'F2',
                27:'F3',
                28:'F4'
            }
            last_round = 24
        else:
            finals_round_map = {
                'F1':24,
                'F2':25,
                'F3':26,
                'F4':27}
            reverse_finals_round_map ={
                24:'F1',
                25:'F2',
                26:'F3',
                27:'F4'
            }
            last_round = 23      

        if 'F' in self.round_id:
            round_num = int(finals_round_map[self.round_id[4:]])
        else:
            round_num = int(self.round_id[4:])
        
        previous_round_num = round_num-1
        if previous_round_num > last_round:
            previous_round = reverse_finals_round_map[previous_round_num]
        else:
            previous_round = str(previous_round_num)
        previous_round = previous_round.zfill(2)

        if round_num == 1:
            previous_season = int(season) - 1
            previous_round = 'F4'
        else:
            previous_season = season
        self.previous_round_id = str(previous_season) + str(previous_round)
        
        return self.previous_round_id
    
@dataclass
class Round:
    round_id: str
    matches: Dict[str, Match]
    
    def __repr__(self) -> str:
        return 'Round({})'.format(self.round_id)
    
@dataclass
class Season:
    season: int
    rounds: Dict[str, Round]
    
    def __repr__(self) -> str:
        return 'Season({})'.format(self.season)

In [4]:
darcy_moore = Player(name = "Darcy Moore", rating = {})
nick_daicos = Player(name = "Nick Daicos")
charlie_cameron = Player(name = "Charlie Cameron")
lachie_neale = Player(name = "Lachie Neale")

In [5]:
darcy_moore

Player(Darcy Moore)

In [6]:
collingwood = Team("Collingwood", {'Darcy Moore': darcy_moore})
brisbane = Team("Brisbane Lions", {'Charlie Cameron': charlie_cameron})
collingwood, brisbane

(Team(Collingwood), Team(Brisbane Lions))

In [7]:
collingwood.players['Darcy Moore'].rating

{}

In [8]:
collingwood_brisbane_2023F4 = Match("2023F4_Collingwood_BrisbaneLions",
                                    collingwood,
                                    brisbane,
                                    "MCG")
                                    # match_summary, player_stats)

In [9]:
collingwood_brisbane_2023F4

Match(Collingwood v Brisbane Lions @ MCG in 2023F4)

In [10]:
round_2023F4 = Round(round_id = '2023F4',
                     matches={collingwood_brisbane_2023F4.match_id: collingwood_brisbane_2023F4})
round_2023F4.matches['2023F4_Collingwood_BrisbaneLions']

Match(Collingwood v Brisbane Lions @ MCG in 2023F4)

In [11]:
season_2023 = Season(season = 2023,
                     rounds = {'2023F4':round_2023F4})
season_2023

Season(2023)

Create Player Database

In [12]:
from typing import Optional

In [285]:
@dataclass
class PlayerDatabase:
    player: Optional[Player]
    
    def add_player(self, name, Player):
        self.__setattr__(name, Player)
        
    def get_player(self, name):
        return self.__getattribute__(name)
    
    def get_players_in_database(self) -> List[str]:
        return [x for x in dir(self) if (("player" not in x) & ("__" not in x))]
    
    def get_player_ratings_dataframe(self) -> pd.DataFrame:
        players_in_database = self.get_players_in_database()
        player_dataframe_list = []
        for player in players_in_database:
            player_rating_dataframe = pd.DataFrame.from_dict(self.get_player(player).rating, orient='index')
            player_rating_dataframe.columns = [player]
            player_dataframe_list.append(player_rating_dataframe.T)
        return pd.concat(player_dataframe_list)

In [280]:
pd.DataFrame.from_dict(player_database.get_player('Christian Petracca').rating, orient='index')

,0
2020F4,4.0000
202101,4.7214
202102,4.9984
202103,5.1032
202104,6.4188
202105,6.1238
202106,6.7278
202107,7.0238
202108,6.7134
202109,6.1050


In [272]:
pd.DataFrame().from_dict({k: v.rating for (k, v) in player_database.get_player('Christian Petracca').rating.items()}, orient = 'index')

AttributeError: 'int' object has no attribute 'rating'

In [271]:
player_database_list = [x for x in dir(player_database) if (("player" not in x) & ("__" not in x))]
player_database_list

['Aaron Francis',
 'Aaron Hall',
 'Aaron Naughton',
 'Aaron vandenBerg',
 'Adam Cerra',
 'Adam Kennedy',
 'Adam Saad',
 'Adam Tomlinson',
 'Adam Treloar',
 'Aidan Corr',
 'Aiden Bonar',
 'Alec Waterman',
 'Alex Davies',
 'Alex Keath',
 'Alex Neal-Bullen',
 'Alex Pearce',
 'Alex Sexton',
 'Alex Witherden',
 'Aliir Aliir',
 'Andrew Brayshaw',
 'Andrew Gaff',
 'Andrew McGrath',
 'Andrew McPherson',
 'Andrew Phillips',
 'Angus Brayshaw',
 'Anthony McDonald-Tipungwuti',
 'Anthony Scott',
 'Anton Tohill',
 'Archie Perkins',
 'Atu Bosenavulagi',
 'Bachar Houli',
 'Bailey Banfield',
 'Bailey Dale',
 'Bailey J. Williams',
 'Bailey Scott',
 'Bailey Smith',
 'Bailey Williams',
 'Bayley Fritsch',
 'Beau McCreery',
 'Ben Ainsworth',
 'Ben Brown',
 'Ben Cavarra',
 'Ben Cunnington',
 'Ben Davis',
 'Ben Keays',
 'Ben King',
 'Ben Long',
 'Ben McEvoy',
 'Ben McKay',
 'Ben Miller',
 'Ben Ronke',
 'Billy Frampton',
 'Blake Acres',
 'Blake Hardwick',
 'Bobby Hill',
 'Boyd Woodcock',
 'Brad Close',
 'Brad 

In [14]:
player_database = PlayerDatabase(player = Player("Player"))
# player_database.add_player('Darcy Moore', Player("Darcy Moore"))

Round 202101

In [15]:
round_id = '202101'

Get Round Matches

In [16]:
def get_round_match_ids(player_stats, round_id):
    
    round_player_stats = player_stats[player_stats['Round_ID'] == round_id]
    
    return list(round_player_stats['Match_ID'].unique())

In [17]:
round_match_ids = get_round_match_ids(player_stats, round_id)
match_id = round_match_ids[-1]
match_id

'202101_WestCoast_GoldCoast'

Get Single Match

In [18]:
def get_match(player_stats, match_id):
    
    return player_stats[player_stats['Match_ID'] == match_id]

In [19]:
match_player_stats = get_match(player_stats, match_id)

Get Teams in Match

In [20]:
import re
def get_teams(match_id):
    
    home_team = re.sub(r"(?<=\w)([A-Z])", r" \1", match_id.split("_")[1])
    away_team = re.sub(r"(?<=\w)([A-Z])", r" \1", match_id.split("_")[-1])
    
    return home_team, away_team

In [21]:
home_team, away_team = get_teams(match_id)

Get Playing Squad for each Match

In [22]:
def get_home_team_squad_list(match_player_stats):
    return list(match_player_stats[match_player_stats['Team'] == match_player_stats['Home_Team']]['Player'].unique())

def get_away_team_squad_list(match_player_stats):
    return list(match_player_stats[match_player_stats['Team'] != match_player_stats['Home_Team']]['Player'].unique())

In [23]:
home_team_squad_list = get_home_team_squad_list(match_player_stats)
away_team_squad_list = get_away_team_squad_list(match_player_stats)

Add New Players to Database

In [36]:
for player in home_team_squad_list + away_team_squad_list:
    if ~hasattr(player_database, player):
        player_database.add_player(player, Player(player, rating={'2020F4':4}))

In [37]:
player_database.get_player("Tim Kelly")

Player(Tim Kelly)

Create Home and Away Teams for Match with defined squads

In [38]:
home_team_squad_dict = {}
for player in home_team_squad_list:
    home_team_squad_dict[player] = player_database.get_player(player)
    
home = Team(home_team, home_team_squad_dict)

away_team_squad_dict = {}
for player in away_team_squad_list:
    away_team_squad_dict[player] = player_database.get_player(player)
    
away = Team(away_team, away_team_squad_dict)

In [39]:
home, away

(Team(West Coast), Team(Gold Coast))

In [40]:
home.get_player_ratings()

,2020F4
Andrew Gaff,4
Brad Sheppard,4
Dom Sheed,4
Jack Darling,4
Jack Petruccelle,4
Jack Redden,4
Jackson Nelson,4
Jamaine Jones,4
Jamie Cripps,4
Jeremy McGovern,4


Get Venue

In [41]:
venue = match_summary[match_summary['Match_ID'] == match_id]['Venue'].iloc[0]

Create Match Object (before Match takes place)

In [42]:
westcoast_goldcoast = Match(match_id, home, away, venue)
westcoast_goldcoast

Match(West Coast v Gold Coast @ Perth Stadium in 202101)

In [43]:
westcoast_goldcoast.home_team.get_player_ratings()

,2020F4
Andrew Gaff,4
Brad Sheppard,4
Dom Sheed,4
Jack Darling,4
Jack Petruccelle,4
Jack Redden,4
Jackson Nelson,4
Jamaine Jones,4
Jamie Cripps,4
Jeremy McGovern,4


Player Projections are Player Ratings from previous round

In [44]:
class Projector:
    def __init__(self,
                 Match: Match,
                 std: int = 35) -> None:
        self.Match = Match
        self.std = std
        
        self.previous_round_id = self.Match.get_previous_round_id()
        
    def calculate_home_team_rating(self):
        home_team_player_ratings = self.Match.home_team.get_player_ratings()
        home_team_player_projection = home_team_player_ratings.loc[:, self.previous_round_id]
        self.home_team_rating = home_team_player_projection.sum()
        
    def get_home_team_rating(self):
        return round(self.home_team_rating, 3)

    def calculate_away_team_rating(self):
        away_team_player_ratings = self.Match.away_team.get_player_ratings()
        away_team_player_projection = away_team_player_ratings.loc[:, self.previous_round_id]
        self.away_team_rating = away_team_player_projection.sum()
    
    def get_away_team_rating(self):
        return round(self.away_team_rating, 3)

In [45]:
match_projector = Projector(westcoast_goldcoast, std = 35)

Sum Home and Away Squad Ratings

In [46]:
match_projector.calculate_home_team_rating()
match_projector.get_home_team_rating()

92

In [47]:
match_projector.calculate_away_team_rating()
match_projector.get_away_team_rating()

92

Create Match Object (with actual match stats)

In [48]:
westcoast_goldcoast = Match(match_id, home, away, venue, match_summary, player_stats)
westcoast_goldcoast

Match(West Coast v Gold Coast @ Perth Stadium in 202101)

In [49]:
westcoast_goldcoast.round_id

'202101'

Get Actual Match Results (player expected VAEP values from match)

In [50]:
westcoast_goldcoast.player_stats.head()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received,Player_Season,Score,xScore_Diff,Home_Team,Away_Team,Opponent,Round,Round_str,Round_ID_num
322,202101_WestCoast_GoldCoast,Gold Coast,Alex Sexton,202101,Alex_Sexton,MEDIUM_FORWARD,CD_I294643,28,186,82,6,RIGHT,QLD,2011.0,2012.0,Springwood (Qld)/Redland (NEAFL),88.0,zone,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1993-12-03,83,1,3.0,0.0,0,1,0.0,0.0,0.0,0,30.8,4,0.0,0.0,0.0,2.0,83.3,12,62,10.0,7.0,1.0,0,0,50.0,2,1,3.0,3,0,0.0,0.0,0.0,6,0.0,1,77.8,0.0,0.0,9,3.0,6,1,1.0,393.0,1,10.0,10.1,1,0.0,6,1.0,2,0.0,0,1,1,0,13,3,9,0,0.0,Interchange,FINAL_TEAM,Interchange,Interchange,2021.0,0.0,2021,6.9863,0.3767,0.0000,1.2053,1.2222,-0.0169,8.8085,9.1348,-0.3264,0.4946,0.0000,1.6381,7.7601,Alex Sexton_2021,7,0.0137,West Coast,Gold Coast,West Coast,1,01,20211
323,202101_WestCoast_GoldCoast,Gold Coast,Ben King,202101,Ben_King,KEY_FORWARD,CD_I1006144,21,202,96,34,RIGHT,VIC,2018.0,2019.0,East Sandr (Vic)/Old Haileybury (Vic)/H&apos;b...,6.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,2000-07-07,89,1,0.0,0.0,0,4,0.0,0.0,0.0,1,33.3,4,3.0,1.0,33.3,1.0,66.7,12,75,8.0,6.0,1.0,0,2,75.0,0,3,1.0,3,0,0.0,0.0,0.0,5,0.0,0,66.7,0.0,0.0,9,3.0,7,3,2.0,281.0,0,8.0,6.4,0,0.0,5,0.0,4,0.0,0,0,0,0,12,4,8,0,0.0,Full Forward,FINAL_TEAM,Key-Forward,Forward,2021.0,0.0,2021,18.6175,0.2611,-0.0156,1.7298,1.8813,-0.1515,7.1877,8.1785,-0.9908,0.9206,0.0000,3.1032,18.8993,Ben King_2021,19,0.3825,West Coast,Gold Coast,West Coast,1,01,20211
324,202101_WestCoast_GoldCoast,Gold Coast,Brandon Ellis,202101,Brandon_Ellis,MIDFIELDER,CD_I293713,28,181,82,4,RIGHT,VIC,2011.0,2012.0,West Coburg (Vic)/Calder U18/Richmond,15.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1993-08-03,87,0,4.0,0.0,0,2,0.0,0.0,0.0,0,15.0,3,0.0,0.0,0.0,10.0,78.9,19,84,15.0,10.0,0.0,1,0,0.0,1,0,2.0,5,0,0.0,0.0,0.0,4,0.0,2,71.4,0.0,0.0,14,2.8,9,0,0.0,292.0,1,14.0,5.4,0,0.0,2,0.0,0,0.0,0,2,0,0,20,3,17,0,0.0,Wing Left,FINAL_TEAM,Wing,Midfield,2021.0,0.0,2021,0.0000,0.3525,-0.0704,0.1027,0.0974,0.0053,1.9506,1.8040,0.1467,0.0418,0.0585,0.4671,5.5437,Brandon Ellis_2021,0,0.0000,West Coast,Gold Coast,West Coast,1,01,20211
325,202101_WestCoast_GoldCoast,Gold Coast,Charlie Ballard,202101,Charlie_Ballard,KEY_DEFENDER,CD_I1008882,22,196,93,10,RIGHT,SA,2017.0,2018.0,Mitcham Hawks (SA)/Sacred Heart College (SA)/S...,42.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1999-07-23,94,0,0.0,0.0,0,5,1.0,33.3,3.0,1,19.0,4,0.0,0.0,0.0,7.0,90.5,21,90,19.0,15.0

Calculate Error

In [51]:
class Evaluator:
    def __init__(self, Match, Projector) -> None:
        
        self.Match = Match
        self.Projector = Projector
        self.home_players = list(self.Match.home_team.players.keys())
        self.away_players = list(self.Match.away_team.players.keys())
        self.player_stats = self.Match.player_stats
        self.player_projections = pd.concat([self.Match.home_team.get_player_ratings(), self.Match.away_team.get_player_ratings()], axis=0)
        self.previous_round_id = self.Match.get_previous_round_id()
        
    def get_player_projection(self, player: str):
        return self.player_projections.loc[player, self.previous_round_id]
    
    def get_player_actual(self, player: str):
        return self.player_stats[self.player_stats['Player'] == player]['exp_vaep_value'].iloc[0]
    
    def get_player_error(self, player):
        return self.get_player_actual(player) - self.get_player_projection(player)
    
    def get_player_absolute_error(self, player):
        return abs(self.get_player_error(player))
    
    def get_match_absolute_error(self):
        match_abs_error = 0
        for player in self.home_players + self.away_players:
            match_abs_error += self.get_player_absolute_error(player)
        return match_abs_error
    
    def get_match_mean_absolute_error(self):
        match_abs_error = 0
        for player in self.home_players + self.away_players:
            match_abs_error += self.get_player_absolute_error(player)
        return match_abs_error / (len(self.home_players) + len(self.away_players))
    
    def evaluate(self):
        
        return round(self.get_match_mean_absolute_error(), 3)

In [52]:
match_evaluator = Evaluator(westcoast_goldcoast, match_projector)
match_evaluator.evaluate()

2.914

Calculate new Player Values

In [53]:
class Updater:
    def __init__(self, Match: Match, Projector: Projector) -> None:
        
        self.Match = Match
        self.Projector = Projector
        self.home_players = list(self.Match.home_team.players.keys())
        self.away_players = list(self.Match.away_team.players.keys())
        self.previous_round_id = self.Match.get_previous_round_id()
        self.player_stats = self.Match.player_stats
        self.player_projections = pd.concat([self.Match.home_team.get_player_ratings(), self.Match.away_team.get_player_ratings()], axis=0)
        
    def get_home_players(self):
        return self.home_players

    def get_away_players(self):
        return self.away_players
    
    def get_player_projection(self, player: str):
        return self.player_projections.loc[player, self.previous_round_id]
    
    def get_player_actual(self, player: str):
        return self.player_stats[self.player_stats['Player'] == player]['exp_vaep_value'].iloc[0]
    
    def calculate_posterior_mean(self, prior_mean: float, actual_mean: float, prior_std: float = 10, actual_std: float = 25):
        return ((actual_std**2 * prior_mean) + ((prior_std**2) * actual_mean)) / (prior_std**2 + actual_std**2)
    
    def calculate_new_rating(self, player, projection_std, actual_std):
        
        projection = self.get_player_projection(player)
        actual = self.get_player_actual(player)
        new_rating = self.calculate_posterior_mean(projection, actual, projection_std, actual_std)
        
        return round(new_rating, 4)

    def update_player_rating(self, player, projection_std = 10, actual_std = 25):
        
        new_rating = self.calculate_new_rating(player, projection_std, actual_std)
        
        if player in self.home_players:
            self.Match.home_team.players[player].add_rating(self.Match.round_id, new_rating)
        if player in self.away_players:
            self.Match.away_team.players[player].add_rating(self.Match.round_id, new_rating) 
                   
    def update(self, projection_std, actual_std):
        
        for player in self.home_players + self.away_players:
            self.update_player_rating(player, projection_std, actual_std)

In [54]:
match_updater = Updater(westcoast_goldcoast, match_projector)
player = "Tim Kelly"
match_updater.get_player_projection(player), match_updater.get_player_actual(player)

(4, 14.1100223402)

Update Player Values (actual v expected)

In [55]:
match_updater.calculate_new_rating(player, 10, 25)

5.3945

In [56]:
match_updater.update_player_rating(player, 10, 25)

In [57]:
player_database.get_player(player).rating

{'2020F4': 4, '202101': 5.3945}

In [58]:
match_updater.update(10, 25)

Initialise player database again

In [286]:
player_database = PlayerDatabase(player = Player("Player"))

Update All Matches in Round 202101

In [227]:
def get_match_information(match_id, player_stats):
    
    print(match_id)
    # Get Match Information
    home_team, away_team = get_teams(match_id)
    match_player_stats = get_match(player_stats, match_id)
    home_team_squad_list = get_home_team_squad_list(match_player_stats)
    away_team_squad_list = get_away_team_squad_list(match_player_stats)
    
    return home_team, away_team, match_player_stats, home_team_squad_list, away_team_squad_list

In [228]:
def add_new_players(home_team_squad_list, away_team_squad_list, previous_round_id):
    
    for player in home_team_squad_list + away_team_squad_list:
        if not hasattr(player_database, player):
            player_database.add_player(player, Player(player, rating={previous_round_id:4}))

In [229]:
def create_team(team_name, team_squad_list):
    
    team_squad_dict = {}
    for player in team_squad_list:
        team_squad_dict[player] = player_database.get_player(player)
            
    return Team(team_name, team_squad_dict)

In [230]:
def get_match_venue(match_summary, match_id):
    return match_summary[match_summary['Match_ID'] == match_id]['Venue'].iloc[0]

In [231]:
def get_round_id(match_id):
    return match_id.split('_')[0]

In [232]:
def get_previous_round_id(round_id):
    
    season = int(round_id[:4])
    if season == 2023:
        finals_round_map = {
            'F1':25,
            'F2':26,
            'F3':27,
            'F4':28}
        reverse_finals_round_map ={
            25:'F1',
            26:'F2',
            27:'F3',
            28:'F4'
        }
        last_round = 24
    else:
        finals_round_map = {
            'F1':24,
            'F2':25,
            'F3':26,
            'F4':27}
        reverse_finals_round_map ={
            24:'F1',
            25:'F2',
            26:'F3',
            27:'F4'
        }
        last_round = 23      

    if 'F' in round_id:
        round_num = int(finals_round_map[round_id[4:]])
    else:
        round_num = int(round_id[4:])
    
    previous_round_num = round_num-1
    if previous_round_num > last_round:
        previous_round = reverse_finals_round_map[previous_round_num]
    else:
        previous_round = str(previous_round_num)
    previous_round = previous_round.zfill(2)

    if round_num == 1:
        previous_season = int(season) - 1
        previous_round = 'F4'
    else:
        previous_season = season
    previous_round_id = str(previous_season) + str(previous_round)
    
    return previous_round_id

In [ ]:
def add_missing_player_ratings(home_team_squad_list, away_team_squad_list, previous_round_id):
    
    for player in home_team_squad_list + away_team_squad_list:
        if (hasattr(player_database, player)):
            available_rounds = list(player_database.get_player(player).rating.keys())
            if previous_round_id not in available_rounds:
                latest_available_round_id = get_latest_available_round_id(available_rounds)
                latest_available_rating = player_database.get_player(player).rating[latest_available_round_id]
                
                latest_available_rating_decayed = latest_available_rating * 0.8
                player_database.get_player(player).add_rating(previous_round_id, latest_available_rating_decayed)

In [248]:
def run_match_scenario(match_id, match_std = 35, projection_std = 10, actual_std = 25):
    
    round_id = get_round_id(match_id)
    previous_round_id = get_previous_round_id(round_id)
    
    print(match_id)
    # Get Match Information
    home_team, away_team = get_teams(match_id)
    match_player_stats = get_match(player_stats, match_id)
    home_team_squad_list = get_home_team_squad_list(match_player_stats)
    away_team_squad_list = get_away_team_squad_list(match_player_stats)
    
    # Set up Players
    add_new_players(home_team_squad_list, away_team_squad_list, previous_round_id)
    
    # Update missing player values
    add_missing_player_ratings(home_team_squad_list, away_team_squad_list, previous_round_id)
    
    # Get Players for each Team        
    home = create_team(home_team, home_team_squad_list)
    away = create_team(away_team, away_team_squad_list)
    
    # Set Venue
    venue = get_match_venue(match_summary, match_id)
    
    # Set up Projections
    match_object = Match(match_id, home, away, venue)
    match_projector = Projector(match_object, std = match_std)
    
    match_projector.calculate_home_team_rating()
    print('Home Team Rating: {}'.format(match_projector.get_home_team_rating()))
    match_projector.calculate_away_team_rating()
    print('Away Team Rating: {}'.format(match_projector.get_away_team_rating()))
    
    # Get Actual Stats
    match_stats_object = Match(match_id, home, away, venue, match_summary, player_stats)
    
    # Calculate Error
    match_evaluator = Evaluator(match_stats_object, match_projector)
    print("Match player MSE: {}".format(match_evaluator.evaluate()))
    
    # Update Player Ratings
    match_updater = Updater(match_stats_object, match_projector)
    match_updater.update(projection_std=projection_std, actual_std=actual_std)

In [249]:
round_id = '202101'
previous_round_id = '2020F4'
round_match_ids = get_round_match_ids(player_stats, round_id)

for match_id in round_match_ids:
    run_match_scenario(match_id, match_std = 35, projection_std = 10, actual_std = 25)

202101_BrisbaneLions_Sydney
Home Team Rating: 92
Away Team Rating: 92
Match player MSE: 3.068
202101_Collingwood_WesternBulldogs
Home Team Rating: 92
Away Team Rating: 92
Match player MSE: 2.76
202101_Essendon_Hawthorn
Home Team Rating: 92
Away Team Rating: 92
Match player MSE: 2.89
202101_GreaterWesternSydney_StKilda
Home Team Rating: 92
Away Team Rating: 92
Match player MSE: 2.611
202101_Melbourne_Fremantle
Home Team Rating: 92
Away Team Rating: 92
Match player MSE: 2.847
202101_NorthMelbourne_PortAdelaide
Home Team Rating: 92
Away Team Rating: 92
Match player MSE: 2.733
202101_Richmond_Carlton
Home Team Rating: 92
Away Team Rating: 92
Match player MSE: 2.946
202101_WestCoast_GoldCoast
Home Team Rating: 92
Away Team Rating: 92
Match player MSE: 2.914


202102

In [250]:
round_id = '202102'
previous_round_id = '202101'
round_match_ids = get_round_match_ids(player_stats, round_id)

for match_id in round_match_ids:
    run_match_scenario(match_id, match_std = 35, projection_std = 10, actual_std = 25)

202102_Carlton_Collingwood
Home Team Rating: 91.187
Away Team Rating: 89.088
Match player MSE: 3.135
202102_Fremantle_GreaterWesternSydney
Home Team Rating: 90.468
Away Team Rating: 94.473
Match player MSE: 2.64
202102_Geelong_BrisbaneLions
Home Team Rating: 92
Away Team Rating: 90.685
Match player MSE: 2.779
202102_GoldCoast_NorthMelbourne
Home Team Rating: 90.511
Away Team Rating: 91.456
Match player MSE: 2.98
202102_Hawthorn_Richmond
Home Team Rating: 92.055
Away Team Rating: 96.227
Match player MSE: 2.388
202102_PortAdelaide_Essendon
Home Team Rating: 95.107
Away Team Rating: 90.873
Match player MSE: 2.499
202102_StKilda_Melbourne
Home Team Rating: 93.14
Away Team Rating: 93.055
Match player MSE: 3.008
202102_Sydney_Adelaide
Home Team Rating: 92.702
Away Team Rating: 92
Match player MSE: 3.432
202102_WesternBulldogs_WestCoast
Home Team Rating: 93.077
Away Team Rating: 94.274
Match player MSE: 3.484


In [254]:
round_id = '202103'
previous_round_id = get_previous_round_id(round_id)
round_match_ids = get_round_match_ids(player_stats, round_id)

for match_id in round_match_ids:
    run_match_scenario(match_id, match_std = 35, projection_std = 10, actual_std = 25)

202103_Adelaide_GoldCoast
Home Team Rating: 95.498
Away Team Rating: 93.699
Match player MSE: 3.256
202103_Carlton_Fremantle
Home Team Rating: 92.16
Away Team Rating: 92.979
Match player MSE: 2.931
202103_Collingwood_BrisbaneLions
Home Team Rating: 93.474
Away Team Rating: 90.513
Match player MSE: 2.72
202103_Essendon_StKilda
Home Team Rating: 89.486
Away Team Rating: 91.52
Match player MSE: 2.493
202103_Geelong_Hawthorn
Home Team Rating: 92.172
Away Team Rating: 89.495
Match player MSE: 2.942
202103_GreaterWesternSydney_Melbourne
Home Team Rating: 91.847
Away Team Rating: 96.502
Match player MSE: 2.693
202103_NorthMelbourne_WesternBulldogs
Home Team Rating: 83.959
Away Team Rating: 95.724
Match player MSE: 3.355
202103_Richmond_Sydney
Home Team Rating: 97.389
Away Team Rating: 94.819
Match player MSE: 2.878
202103_WestCoast_PortAdelaide
Home Team Rating: 94.607
Away Team Rating: 96.744
Match player MSE: 2.615


In [255]:
round_id = '202104'
previous_round_id = get_previous_round_id(round_id)
round_match_ids = get_round_match_ids(player_stats, round_id)

for match_id in round_match_ids:
    run_match_scenario(match_id, match_std = 35, projection_std = 10, actual_std = 25)

202104_Collingwood_GreaterWesternSydney
Home Team Rating: 94.163
Away Team Rating: 88.311
Match player MSE: 2.595
202104_Fremantle_Hawthorn
Home Team Rating: 89.159
Away Team Rating: 91.037
Match player MSE: 2.65
202104_GoldCoast_Carlton
Home Team Rating: 93.15
Away Team Rating: 96.196
Match player MSE: 2.934
202104_Melbourne_Geelong
Home Team Rating: 97.589
Away Team Rating: 91.885
Match player MSE: 2.838
202104_NorthMelbourne_Adelaide
Home Team Rating: 79.409
Away Team Rating: 94.639
Match player MSE: 2.994
202104_PortAdelaide_Richmond
Home Team Rating: 99.279
Away Team Rating: 91.574
Match player MSE: 2.821
202104_StKilda_WestCoast
Home Team Rating: 87.872
Away Team Rating: 95.175
Match player MSE: 3.298
202104_Sydney_Essendon
Home Team Rating: 95.15
Away Team Rating: 93.373
Match player MSE: 2.733
202104_WesternBulldogs_BrisbaneLions
Home Team Rating: 103.072
Away Team Rating: 91.702
Match player MSE: 2.858


In [287]:
season_2021 = list(player_stats[player_stats['Season'] == 2021]['Round_ID'].unique())
for round_id in season_2021:
    
    previous_round_id = get_previous_round_id(round_id)
    round_match_ids = get_round_match_ids(player_stats, round_id)

    for match_id in round_match_ids:
        run_match_scenario(match_id, match_std = 35, projection_std = 10, actual_std = 25)

202101_BrisbaneLions_Sydney
Home Team Rating: 92
Away Team Rating: 92
Match player MSE: 3.068
202101_Collingwood_WesternBulldogs
Home Team Rating: 92
Away Team Rating: 92
Match player MSE: 2.76
202101_Essendon_Hawthorn
Home Team Rating: 92
Away Team Rating: 92
Match player MSE: 2.89
202101_GreaterWesternSydney_StKilda
Home Team Rating: 92
Away Team Rating: 92
Match player MSE: 2.611
202101_Melbourne_Fremantle
Home Team Rating: 92
Away Team Rating: 92
Match player MSE: 2.847
202101_NorthMelbourne_PortAdelaide
Home Team Rating: 92
Away Team Rating: 92
Match player MSE: 2.733
202101_Richmond_Carlton
Home Team Rating: 92
Away Team Rating: 92
Match player MSE: 2.946
202101_WestCoast_GoldCoast
Home Team Rating: 92
Away Team Rating: 92
Match player MSE: 2.914
202102_Carlton_Collingwood
Home Team Rating: 91.187
Away Team Rating: 89.088
Match player MSE: 3.135
202102_Fremantle_GreaterWesternSydney
Home Team Rating: 90.468
Away Team Rating: 94.473
Match player MSE: 2.64
202102_Geelong_BrisbaneLi

In [290]:
player_database.get_player_ratings_dataframe().sort_values(by='2021F4', ascending = False).head(10)

,2020F4,202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112,202119,202120,202121,202122,202123,2021F1,202113,202114,202115,202116,202117,202118,2021F2,2021F3,2021F4
Christian Petracca,4.0,4.7214,4.9984,5.1032,6.4188,6.1238,6.7278,7.0238,6.7134,6.1050,6.5445,7.2797,7.9718,7.6684,7.3708,8.0176,7.6501,6.5498,7.4082,7.9023,6.3218,5.9432,5.8171,7.6166,7.7317,5.9266,7.1925,8.1757
Jack Macrae,4.0,4.2869,6.4157,6.9569,7.8714,8.0600,9.1634,8.0082,8.4722,8.2540,9.0044,9.3355,8.3686,6.9273,7.2086,8.3176,7.8837,7.4541,8.6486,6.6949,6.4634,6.4618,6.6283,6.5964,7.3161,9.1861,9.0866,7.6908
Marcus Bontempelli,4.0,4.0039,5.3814,5.4942,6.2469,6.6551,7.8715,8.4442,8.5890,8.8886,10.2204,9.1357,9.4658,7.5003,7.5022,7.7806,7.2558,7.2012,6.7012,7.5726,7.4210,7.6946,7.6653,7.2409,6.8561,7.2518,7.2878,7.3518
Clayton Oliver,4.0,3.9770,4.9202,5.0432,5.2863,4.7267,5.2783,5.7890,6.1696,6.0764,7.8651,8.4708,8.7424,6.8968,8.4151,8.0704,7.3830,8.0598,8.9270,8.7843,7.0274,6.6614,6.2104,6.8044,6.9454,7.1416,6.8727,6.9466
Max Gawn,4.0,4.0264,4.1088,4.8379,4.7226,5.4932,4.9580,4.3655,4.2079,4.3600,4.7810,5.1869,6.2745,4.5679,5.7609,5.5914,6.1271,6.5300,6.4642,5.1414,4.1131,4.2770,4.8456,4.8082,4.3943,5.1714,7.2695,6.5124
Tom Liberatore,4.0,4.3886,5.3919,6.2437,5.6206,5.6254,5.7027,5.8001,6.2388,6.0261,6.7383,6.2238,6.5662,5.4473,5.8057,5.2492,5.7915,5.2664,5.0958,5.2530,6.3020,6.3639,6.9662,6.4961,5.1969,5.4285,5.3987,6.0707
Adam Treloar,4.0,4.0784,4.9032,5.4367,5.2533,5.3833,5.8790,5.7413,5.5464,5.9030,5.2408,NaN,NaN,NaN,4.1926,4.1070,3.5982,3.4249,4.6742,NaN,NaN,NaN,NaN,NaN,NaN,4.5125,5.4399,5.7549
Bayley Fritsch,4.0,4.2144,4.3212,4.4700,5.0948,4.0758,3.9599,4.3399,3.5750,3.9994,3.9707,4.8775,4.8278,3.1245,2.9376,3.2192,4.7045,4.7251,5.1265,4.2310,3.3848,3.3747,3.7331,3.3970,3.8030,4.1012,4.8393,5.0283
Jake Lever,4.0,4.2425,4.5650,4.1912,4.4867,4.5151,4.6138,4.5748,4.7569,4.3004,4.0745,5.1855,4.6982,5.1788,5.4063,5.4725,5.5296,6.4255,6.5919,4.2462,3.3970,3.8737,4.4790,4.0994,4.9471,5.2735,5.4293,4.4961
Bailey Smith,4.0,4.3522,4.0087,4.3121,4.5918,5.0762,4.9952,4.5449,4.7111,5.2790,5.5377,5.5230,5.5687,4.5542,4.1570,4.1444,4.1091,4.3030,3.8697,4.4550,4.3455,4.9950,4.6716,5.5268,4.4837,3.5354,4.6415,4.4819


In [261]:
player_database.get_player("Marcus Bontempelli").rating

{'2020F4': 4,
 '202101': 4.0039,
 '202102': 5.3814,
 '202103': 5.4942,
 '202104': 6.2469,
 '202105': 6.6551,
 '202106': 7.8715,
 '202107': 8.4442,
 '202108': 8.589,
 '202109': 8.8886,
 '202110': 10.2204,
 '202111': 9.1357,
 '202112': 9.4658,
 '202113': 7.57264,
 '202114': 7.421,
 '202115': 7.6946,
 '202116': 7.6653,
 '202117': 7.2409,
 '202118': 6.8561,
 '202119': 7.5003,
 '202120': 7.5022,
 '202121': 7.7806,
 '202122': 7.2558,
 '202123': 7.2012,
 '2021F1': 6.7012,
 '2021F2': 7.2518,
 '2021F3': 7.2878,
 '2021F4': 7.3518}